In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm

In [0]:
train = pd.read_csv("/content/drive/My Drive/Machine _Hack/janta-hack-CV/train/train.csv")

In [17]:
train.head()

,image_names,emergency_or_not
0,1503.jpg,0
1,1420.jpg,0
2,1764.jpg,0
3,1356.jpg,0
4,1117.jpg,0


In [37]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img("/content/drive/My Drive/Machine _Hack/janta-hack-CV/train/images/"+train['image_names'][i], grayscale=False)
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 1646/1646 [00:04<00:00, 383.09it/s]


In [0]:
y=train['emergency_or_not'].values
y = to_categorical(y)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 294)

In [44]:
from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.applications import VGG16
from keras.models import Model

image_size = 224
input_shape = (image_size, image_size, 3)

epochs = 5
batch_size = 16

pre_trained_model = VGG16(input_shape=input_shape, include_top=False, weights="imagenet")
    
for layer in pre_trained_model.layers[:15]:
    layer.trainable = False

for layer in pre_trained_model.layers[15:]:
    layer.trainable = True
    
last_layer = pre_trained_model.get_layer('block5_pool')
last_output = last_layer.output
    
# Flatten the output layer to 1 dimension
x = GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(2, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [0]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=200)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

In [48]:
model.fit(X_train, y_train, epochs=50,  validation_data=(X_test, y_test),callbacks=[es,mc])

Train on 1399 samples, validate on 247 samples
Epoch 1/50
1399/1399 [==============================] - 9s 7ms/step - loss: 0.0079 - accuracy: 0.9989 - val_loss: 0.2795 - val_accuracy: 0.9069

Epoch 00001: val_accuracy improved from -inf to 0.90688, saving model to best_model.h5
Epoch 2/50
1399/1399 [==============================] - 10s 7ms/step - loss: 0.0098 - accuracy: 0.9982 - val_loss: 0.2834 - val_accuracy: 0.9049

Epoch 00002: val_accuracy did not improve from 0.90688
Epoch 3/50
1399/1399 [==============================] - 10s 7ms/step - loss: 0.0103 - accuracy: 0.9982 - val_loss: 0.2900 - val_accuracy: 0.9109

Epoch 00003: val_accuracy improved from 0.90688 to 0.91093, saving model to best_model.h5
Epoch 4/50
1399/1399 [==============================] - 10s 7ms/step - loss: 0.0100 - accuracy: 0.9986 - val_loss: 0.2828 - val_accuracy: 0.9069

Epoch 00004: val_accuracy did not improve from 0.91093
Epoch 5/50
1399/1399 [==============================] - 9s 7ms/step - loss: 0.0114 

In [0]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',kernel_initializer='he_uniform',input_shape=(28,28,1)))
# model.add(Conv2D(64, (3, 3), activation='relu',kernel_initializer='he_uniform'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(2, activation='softmax'))

In [0]:
# model.compile(loss='binary_crossentropy',optimizer="sgd",metrics=['accuracy'])

In [0]:
# model.fit(X_train, y_train, epochs=500,  validation_data=(X_test, y_test))

Train on 1399 samples, validate on 247 samples
Epoch 1/500
1399/1399 [==============================] - 8s 5ms/step - loss: 0.7981 - accuracy: 0.5611 - val_loss: 0.6766 - val_accuracy: 0.6235
Epoch 2/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.6777 - accuracy: 0.5890 - val_loss: 0.6571 - val_accuracy: 0.6680
Epoch 3/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.6658 - accuracy: 0.5854 - val_loss: 0.6462 - val_accuracy: 0.6518
Epoch 4/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.6589 - accuracy: 0.6133 - val_loss: 0.6273 - val_accuracy: 0.6842
Epoch 5/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.6599 - accuracy: 0.6097 - val_loss: 0.6206 - val_accuracy: 0.6518
Epoch 6/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.6253 - accuracy: 0.6505 - val_loss: 0.6421 - val_accuracy: 0.6397
Epoch 7/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.6186

Epoch 57/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.3528 - accuracy: 0.8463 - val_loss: 0.4647 - val_accuracy: 0.8016
Epoch 58/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.3497 - accuracy: 0.8513 - val_loss: 0.4484 - val_accuracy: 0.8057
Epoch 59/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.3525 - accuracy: 0.8413 - val_loss: 0.4872 - val_accuracy: 0.7935
Epoch 60/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.3468 - accuracy: 0.8528 - val_loss: 0.4859 - val_accuracy: 0.7773
Epoch 61/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.3344 - accuracy: 0.8549 - val_loss: 0.4635 - val_accuracy: 0.7733
Epoch 62/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.3242 - accuracy: 0.8628 - val_loss: 0.4517 - val_accuracy: 0.7814
Epoch 63/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.3298 - accuracy: 0.8599 - val_loss: 0.4666 -

1399/1399 [==============================] - 5s 3ms/step - loss: 0.1435 - accuracy: 0.9550 - val_loss: 0.4937 - val_accuracy: 0.8097
Epoch 114/500
1399/1399 [==============================] - 5s 4ms/step - loss: 0.1311 - accuracy: 0.9514 - val_loss: 0.5248 - val_accuracy: 0.8016
Epoch 115/500
1399/1399 [==============================] - 5s 4ms/step - loss: 0.1353 - accuracy: 0.9464 - val_loss: 0.5044 - val_accuracy: 0.8057
Epoch 116/500
1399/1399 [==============================] - 5s 3ms/step - loss: 0.1287 - accuracy: 0.9543 - val_loss: 0.5196 - val_accuracy: 0.7935
Epoch 117/500
1399/1399 [==============================] - 5s 4ms/step - loss: 0.1480 - accuracy: 0.9392 - val_loss: 0.4909 - val_accuracy: 0.7976
Epoch 118/500
1399/1399 [==============================] - 6s 4ms/step - loss: 0.1286 - accuracy: 0.9521 - val_loss: 0.4962 - val_accuracy: 0.7935
Epoch 119/500
1399/1399 [==============================] - 5s 4ms/step - loss: 0.1363 - accuracy: 0.9457 - val_loss: 0.4962 - val_ac

Epoch 169/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0577 - accuracy: 0.9828 - val_loss: 0.6352 - val_accuracy: 0.7976
Epoch 170/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0542 - accuracy: 0.9814 - val_loss: 0.6282 - val_accuracy: 0.8219
Epoch 171/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0671 - accuracy: 0.9821 - val_loss: 0.6223 - val_accuracy: 0.7935
Epoch 172/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0635 - accuracy: 0.9807 - val_loss: 0.6108 - val_accuracy: 0.8016
Epoch 173/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0600 - accuracy: 0.9814 - val_loss: 0.6041 - val_accuracy: 0.8016
Epoch 174/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0581 - accuracy: 0.9814 - val_loss: 0.6298 - val_accuracy: 0.7976
Epoch 175/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0574 - accuracy: 0.9771 - val_loss: 0

Epoch 225/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0294 - accuracy: 0.9907 - val_loss: 0.7150 - val_accuracy: 0.8097
Epoch 226/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0419 - accuracy: 0.9871 - val_loss: 0.7063 - val_accuracy: 0.8016
Epoch 227/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0390 - accuracy: 0.9871 - val_loss: 0.7094 - val_accuracy: 0.8097
Epoch 228/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0426 - accuracy: 0.9850 - val_loss: 0.6733 - val_accuracy: 0.8016
Epoch 229/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0291 - accuracy: 0.9921 - val_loss: 0.6851 - val_accuracy: 0.8057
Epoch 230/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0326 - accuracy: 0.9900 - val_loss: 0.7081 - val_accuracy: 0.8057
Epoch 231/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0327 - accuracy: 0.9871 - val_loss: 0

Epoch 281/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0268 - accuracy: 0.9907 - val_loss: 0.7841 - val_accuracy: 0.8097
Epoch 282/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0245 - accuracy: 0.9936 - val_loss: 0.7752 - val_accuracy: 0.8057
Epoch 283/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0185 - accuracy: 0.9943 - val_loss: 0.7897 - val_accuracy: 0.8057
Epoch 284/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0276 - accuracy: 0.9907 - val_loss: 0.8193 - val_accuracy: 0.8097
Epoch 285/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0193 - accuracy: 0.9936 - val_loss: 0.8205 - val_accuracy: 0.8138
Epoch 286/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0173 - accuracy: 0.9971 - val_loss: 0.8052 - val_accuracy: 0.8219
Epoch 287/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0210 - accuracy: 0.9936 - val_loss: 0

Epoch 337/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0126 - accuracy: 0.9971 - val_loss: 0.8476 - val_accuracy: 0.7976
Epoch 338/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0183 - accuracy: 0.9950 - val_loss: 0.8330 - val_accuracy: 0.7976
Epoch 339/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0215 - accuracy: 0.9929 - val_loss: 0.8090 - val_accuracy: 0.8016
Epoch 340/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0104 - accuracy: 0.9993 - val_loss: 0.8426 - val_accuracy: 0.8016
Epoch 341/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0214 - accuracy: 0.9957 - val_loss: 0.8104 - val_accuracy: 0.8057
Epoch 342/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0125 - accuracy: 0.9964 - val_loss: 0.8364 - val_accuracy: 0.8016
Epoch 343/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0135 - accuracy: 0.9957 - val_loss: 0

Epoch 393/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0161 - accuracy: 0.9950 - val_loss: 0.8544 - val_accuracy: 0.8057
Epoch 394/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0185 - accuracy: 0.9964 - val_loss: 0.8531 - val_accuracy: 0.8057
Epoch 395/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0109 - accuracy: 0.9979 - val_loss: 0.8755 - val_accuracy: 0.8057
Epoch 396/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0110 - accuracy: 0.9971 - val_loss: 0.8759 - val_accuracy: 0.8057
Epoch 397/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0125 - accuracy: 0.9971 - val_loss: 0.8979 - val_accuracy: 0.8097
Epoch 398/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0192 - accuracy: 0.9950 - val_loss: 0.8627 - val_accuracy: 0.7854
Epoch 399/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0206 - accuracy: 0.9914 - val_loss: 0

Epoch 449/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0107 - accuracy: 0.9957 - val_loss: 0.8937 - val_accuracy: 0.8097
Epoch 450/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0133 - accuracy: 0.9943 - val_loss: 0.9127 - val_accuracy: 0.8097
Epoch 451/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0109 - accuracy: 0.9957 - val_loss: 0.9129 - val_accuracy: 0.8097
Epoch 452/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0107 - accuracy: 0.9950 - val_loss: 0.9089 - val_accuracy: 0.8057
Epoch 453/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0117 - accuracy: 0.9957 - val_loss: 0.8958 - val_accuracy: 0.7976
Epoch 454/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0106 - accuracy: 0.9964 - val_loss: 0.9099 - val_accuracy: 0.8097
Epoch 455/500
1399/1399 [==============================] - 4s 3ms/step - loss: 0.0108 - accuracy: 0.9950 - val_loss: 0

In [0]:
test = pd.read_csv("/content/drive/My Drive/Machine _Hack/janta-hack-CV/test/test.csv")

In [53]:
test_image = []
for i in tqdm(range(test.shape[0])):
    img = image.load_img('/content/drive/My Drive/Machine _Hack/janta-hack-CV/test/images/'+test['image_names'][i])
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)
test = np.array(test_image)


100%|██████████| 706/706 [06:43<00:00,  1.75it/s]


In [0]:
from keras.models import load_model
model = load_model("/content/best_model.h5")

In [0]:
prediction = model.predict(test)

In [0]:
classs = np.argmax(prediction, axis = 1)

In [0]:
sample = pd.read_csv("/content/drive/My Drive/Machine _Hack/janta-hack-CV/sample_submission_yxjOnvz.csv")

In [0]:
sample['emergency_or_not'] = classs
sample.to_csv("Final_sub_vgg"+'.csv', header=True, index=False)
